In [2]:
word_set = set()
tags_dict = {'START': 0, 'END': 0}
tags_count = {}
emission_dict = {}
transition_dict = {}

file_path = "./hmm-training-data/it_isdt_train_tagged.txt"

In [3]:
def word_preprocess(word):
    return word.lower()
    # return word

In [4]:
with open(file_path, encoding='utf-8') as input_file:
    lines = input_file.readlines()
    for each_line in lines:
        tokens = each_line.split()

        previous_tag = "START"

        for each_token in tokens:
            token_split = each_token.rsplit("/", 1)
            word = word_preprocess(token_split[0])
            tag = token_split[1]

            # word
            word_set.add(word)

            # tags_dict
            if tag not in tags_dict:
                tags_dict[tag] = 1
            else:
                tags_dict[tag] += 1

            # transition_dict
            transition_tuple = (previous_tag, tag)
            if transition_tuple not in transition_dict:
                transition_dict[transition_tuple] = 1
            else:
                transition_dict[transition_tuple] += 1
            previous_tag = tag

            # emission_dict
            emission_tuple = (tag, word)
            if emission_tuple not in emission_dict:
                emission_dict[emission_tuple] = 1
            else:
                emission_dict[emission_tuple] += 1

        if (tag, 'END') not in transition_dict:
            transition_dict[(tag, 'END')] = 1
        else:
            transition_dict[(tag, 'END')] += 1

        tags_dict['START'] += 1
        tags_dict['END'] += 1

In [5]:
for tag in tags_dict:
    tags_count[tag] = tags_dict[tag]

# Smoothing
for tag_1 in tags_dict:
    for tag_2 in tags_dict:
        if tag_1 == "START" and tag_2 == "END":
            continue
        if tag_1 == "END":
            continue
        if tag_2 == "START":
            continue

        if (tag_1, tag_2) not in transition_dict:
            transition_dict[(tag_1, tag_2)] = 1
            tags_count[tag_1] += 1

assert ('START', 'END') not in transition_dict

In [6]:
# Transition dictionary length = number of tags ^ 2 + ('START', tags) + (tags, 'END')
assert len(transition_dict) == (len(tags_dict) - 2) ** 2 + 2 * (len(tags_dict) - 2)

In [7]:
for transition in transition_dict:
    transition_dict[transition] = transition_dict[transition] / tags_count[transition[0]]
    assert transition_dict[transition] <= 1, 'Something wrong with smoothing transition probabilities'

In [8]:
for emission in emission_dict:
    tag = emission[0]
    emission_dict[emission] = emission_dict[emission] / tags_dict[tag]
    assert emission_dict[emission] <= 1, 'Something wrong with emission probabilities'

In [9]:
with open('hmmmodel.txt', 'w', encoding='utf-8') as f:
    f.truncate(0)
    f.write(f'{word_set}\n')
    f.write(f'{tags_dict}\n')
    f.write(f'{transition_dict}\n')
    f.write(f'{emission_dict}\n')